# Índice
<p>
<div class="lev1 toc-item">
    <a href="#Extracción-de-transacciones-de-2016" data-toc-modified-id="Extracción-de-transacciones-de-2016">
        <span class="toc-item-num">1&nbsp;&nbsp;</span>
        Extracción de transacciones de 2016</a></div>

<div class="lev1 toc-item">
    <a href="#Limpieza-y-transformación-de-transacciones" data-toc-modified-id="Limpieza-y-transformación-de-transacciones">
    <span class="toc-item-num">2&nbsp;&nbsp;</span>
    Limpieza y transformación de transacciones</a></div>
    
<div class="lev1 toc-item">
    <a href="#Extracción-de-datos-de-capacidad-de-plazas" data-toc-modified-id="Extracción-de-datos-de-capacidad-de-plazas">
    <span class="toc-item-num">3&nbsp;&nbsp;</span>
    Extracción de datos de capacidad de plazas</a></div>
    
<div class="lev1 toc-item">
    <a href="#Extracción-de-datos-de-ubicación-geográfica" data-toc-modified-id="Extracción-de-datos-de-ubicación-geográfica">
    <span class="toc-item-num">4&nbsp;&nbsp;</span>
    Extracción de datos de ubicación geográfica</a></div>
    
<div class="lev1 toc-item">
    <a href="#Relación-de-transacciones,-ubicaciones-y-capacidad" data-toc-modified-id="Relación-de-transacciones,-ubicaciones-y-capacidad">
    <span class="toc-item-num">5&nbsp;&nbsp;</span>
    Relación de transacciones, ubicaciones y capacidad</a></div>
          
<div class="lev1 toc-item">
    <a href="#Construcción-de-la-serie-de-ocupación-por-timestamp" data-toc-modified-id="Construcción-de-la-serie-de-ocupación-por-timestamp">
    <span class="toc-item-num">6&nbsp;&nbsp;</span>
    Construcción de la serie de ocupación por timestamp</a></div>

<div class="lev1 toc-item">
    <a href="#Corrección-de-la-capacidad-de-plazas-disponibles" data-toc-modified-id="Corrección-de-la-capacidad-de-plazas-disponibles">
    <span class="toc-item-num">7&nbsp;&nbsp;</span>
    Corrección de la capacidad de plazas disponibles</a></div>

<div class="lev1 toc-item">
    <a href="#Completamos-la-serie-con-las-horas-intermedias-faltantes" data-toc-modified-id="Completamos-la-serie-con-las-horas-intermedias-faltantes">
    <span class="toc-item-num">8&nbsp;&nbsp;</span>
    Completamos la serie con las horas intermedias faltantes</a></div>

<div class="lev1 toc-item">
    <a href="#Completamos-la-serie-con-variables-adicionales" data-toc-modified-id="Completamos-la-serie-con-variables-adicionales">
    <span class="toc-item-num">9&nbsp;&nbsp;</span>
    Completamos la serie con variables adicionales</a></div>
    
<div class="lev1 toc-item">
    <a href="#Filtramos-la-serie" data-toc-modified-id="Filtramos-la-serie">
    <span class="toc-item-num">10&nbsp;&nbsp;</span>
    Filtramos la serie</a></div>

In [1]:
import numpy as np
import pandas as pd
import os
import operator
import seaborn as sns
% matplotlib inline

## Extracción de transacciones de 2016

In [2]:
df = pd.read_csv('./data/ParkingTransaction_2016_cleaned.csv', parse_dates=['TransactionDateTime'])

In [3]:
df['TransactionDateTime'].describe()

count                10935395
unique                5505469
top       2016-04-08 12:03:59
freq                       63
first     2016-01-01 11:49:15
last      2016-12-31 20:00:51
Name: TransactionDateTime, dtype: object

In [4]:
init_transactions = df['TransactionDateTime'].count()

In [5]:
import re

_underscorer1 = re.compile(r'(.)([A-Z][a-z]+)')
_underscorer2 = re.compile('([a-z0-9])([A-Z])')

def camelToSnake(s):
    subbed = _underscorer1.sub(r'\1_\2', s)
    return _underscorer2.sub(r'\1_\2', subbed).lower()

In [6]:
df.columns = df.columns.map(lambda x: camelToSnake(x))

df.sort_values('transaction_date_time', inplace=True)

# creamos nueva columna 'final_date_time'
df['final_date_time'] = df.transaction_date_time.add(pd.to_timedelta(df.duration_mins, unit="m"))

df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10935395 entries, 8 to 10912795
Data columns (total 12 columns):
unnamed: 0               int64
transaction_id           int64
transaction_date_time    datetime64[ns]
transaction_date         object
time_start               object
time_expired             object
duration_mins            int64
amount                   float64
payment_mean             object
meter_code               int64
element_key              int64
final_date_time          datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(5), object(4)
memory usage: 3.4 GB


In [15]:
df = df.drop(columns=['unnamed: 0','transaction_date','amount','payment_mean',
                      'meter_code','time_start','time_expired'])

In [16]:
df.head()

,transaction_id,transaction_date_time,duration_mins,element_key,final_date_time
8,183506786,2016-01-01 11:49:15,0,54730,2016-01-01 11:49:15
4,183511748,2016-01-01 12:36:04,0,88773,2016-01-01 12:36:04
17,183511753,2016-01-01 12:36:08,0,88773,2016-01-01 12:36:08
15,183529739,2016-01-01 12:40:30,0,57354,2016-01-01 12:40:30
12,183527118,2016-01-01 14:52:45,0,76102,2016-01-01 14:52:45


In [17]:
df.tail()

,transaction_id,transaction_date_time,duration_mins,element_key,final_date_time
10909957,313159887,2016-12-31 19:58:49,2,81194,2016-12-31 20:00:49
10918741,313159702,2016-12-31 19:58:53,2,79570,2016-12-31 20:00:53
10910192,313159980,2016-12-31 19:59:24,1,1234,2016-12-31 20:00:24
10907062,313159989,2016-12-31 19:59:45,0,43914,2016-12-31 19:59:45
10912795,313160195,2016-12-31 20:00:51,0,8306,2016-12-31 20:00:51


### Limpieza y transformación de transacciones

In [18]:
df.transaction_id.duplicated().sum()  # no hay transacciones con id duplicado

0

In [19]:
df.isnull().sum()  # no hay valores nulos en el dataset

transaction_id           0
transaction_date_time    0
duration_mins            0
element_key              0
final_date_time          0
dtype: int64

#### Duración incorrecta

In [20]:
# Observamos algunas transacciones con duración = 0
df.duration_mins.loc[df.duration_mins == 0].count()

9086

In [21]:
# Incluso hay algunas transacciones con duración negativa
df.duration_mins.loc[df.duration_mins < 0].count()

1118

In [22]:
# Eliminamos ambos casos que suponen menos de un 0.1% del total
df = df.loc[df.duration_mins > 0]
df = df.drop(columns=['duration_mins'])

#### Transacciones de larga duración

In [23]:
# Procesado de transacciones con distinta fecha de inicio y fin
long_trans = df.loc[df.transaction_date_time.dt.date != df.final_date_time.dt.date]
print(long_trans.shape)
indexes = df.loc[df.transaction_date_time.dt.date != df.final_date_time.dt.date].index.get_values()
df = df.drop(index=indexes, axis=1)

(13403, 4)


In [24]:
long_trans_dup = long_trans.copy()
long_trans_dup = long_trans_dup.loc[long_trans_dup.final_date_time.dt.hour >= 8]
long_trans.final_date_time = long_trans.transaction_date_time.apply(
    lambda x: x.replace(hour=20,minute=0,second=0,microsecond=0))
long_trans_dup.transaction_date_time = long_trans_dup.final_date_time.apply(
    lambda x: x.replace(hour=8,minute=0,second=0,microsecond=0))

In [25]:
long_trans_full = pd.concat([long_trans,long_trans_dup], ignore_index=True, sort=True)
df = pd.concat([df,long_trans_full], ignore_index=True, sort=True)

#### Horario de funcionamiento de los parquímetros

In [26]:
# Eliminamos las transacciones con inicio y fin antes de las 08:00
indexes = df.loc[(df.transaction_date_time.dt.hour < 8) & (df.final_date_time.dt.hour < 8)].index.get_values()
print(indexes.size)
df = df.drop(index=indexes, axis=1)

55413


In [27]:
# Eliminamos las transacciones con inicio y fin después de las 20:00
indexes = df.loc[(df.transaction_date_time.dt.hour >= 20) & (df.final_date_time.dt.hour >= 20)].index.get_values()
print(indexes.size)
df = df.drop(index=indexes, axis=1)

17107


In [28]:
# Redondeamos la hora (sin minutos o segundos) y fijamos a 08:00 la hora mínima de comienzo
df.transaction_date_time = np.where(df.transaction_date_time.dt.hour < 8, 
                                    df.transaction_date_time.apply(lambda x: x.replace(hour=8,minute=0,second=0,microsecond=0)), 
                                    df.transaction_date_time.apply(lambda x: x.replace(minute=0,second=0,microsecond=0)))

In [29]:
# Redondeamos la hora final (sin minutos o segundos) y fijamos a 20:00 la hora máxima final
df.final_date_time = np.where(df.final_date_time.dt.hour > 20,
                              df.final_date_time.apply(lambda x: x.replace(hour=20,minute=0,second=0,microsecond=0)), 
                              df.final_date_time.apply(lambda x: x.replace(minute=0,second=0,microsecond=0)))

In [30]:
# Comprobamos que no hay transacciones fuera del rango horario de uso de los parquímetros (de 8 a 20h)
df.element_key.loc[((df.transaction_date_time.dt.hour > 20) | (df.transaction_date_time.dt.hour < 8)) | 
       ((df.final_date_time.dt.hour > 20) | (df.final_date_time.dt.hour < 8))].count()

0

#### Domingos

In [35]:
# Observamos algunas transacciones realizadas por error en domingo
df.transaction_date_time.loc[df.transaction_date_time.dt.weekday == 6].count()

0

In [36]:
# Las eliminamos también
df = df.loc[df.transaction_date_time.dt.weekday != 6]

In [37]:
df.shape  

(10855107, 4)

In [38]:
# Con la limpieza hemos reducido el tamaño del dataset inicial en un 0.75%
(init_transactions-(df.shape[0]-long_trans_dup.shape[0]))/init_transactions*100

0.7586648676156645

In [29]:
df.to_csv('./data/ParkingTransaction_2016_cleaned_ourETL.csv', index=False)

## Extracción de datos de capacidad de plazas

In [39]:
blocks = pd.read_csv('./data/Blockface_cleaned.csv')

In [40]:
blocks.shape

(13706, 39)

In [41]:
blocks.columns = blocks.columns.map(lambda x: camelToSnake(x))

In [42]:
blocks.loc[blocks.element_key == 1001]

,pay_station_blockface_id,element_key,parking_spaces,paid_parking_area,parking_time_limit_category,peak_hour_start1,peak_hour_end1,peak_hour_start2,peak_hour_end2,paid_area_start_time,...,saturday_start1,saturday_end1,saturday_rate2,saturday_start2,saturday_end2,saturday_rate3,saturday_start3,saturday_end3,start_time_saturday,end_time_saturday
469,7576,1001,5.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,08:00:00,11:00:00,4.0,11:00:00,18:00:00,NaN,NaN,NaN,08:00:00,18:00:00
2783,10071,1001,5.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,08:00:00,11:00:00,4.5,11:00:00,18:00:00,NaN,NaN,NaN,08:00:00,18:00:00
3017,10262,1001,4.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,08:00:00,11:00:00,4.5,11:00:00,18:00:00,NaN,NaN,NaN,08:00:00,18:00:00
4648,11976,1001,4.0,NaN,120.0,06:00:00,09:00:00,15:00:00,18:00:00,NaN,...,00:00:00,00:00:00,0.0,00:00:00,00:00:00,0.0,00:00:00,00:00:00,NaN,NaN
5989,13670,1001,4.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,08:00:00,11:00:00,4.5,11:00:00,18:00:00,NaN,NaN,NaN,08:00:00,18:00:00
6301,14348,1001,5.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,08:00:00,11:00:00,5.0,11:00:00,18:00:00,NaN,NaN,NaN,08:00:00,18:00:00
7077,13939,1001,5.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,08:00:00,11:00:00,4.5,11:00:00,18:00:00,NaN,NaN,NaN,08:00:00,18:00:00
8059,3405,1001,5.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10576,1352,1001,7.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11332,6046,1001,5.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,08:00:00,11:00:00,3.5,11:00:00,18:00:00,NaN,NaN,NaN,08:00:00,18:00:00


In [43]:
blocks.isnull().sum()

pay_station_blockface_id           0
element_key                        0
parking_spaces                    14
paid_parking_area               1831
parking_time_limit_category     1330
peak_hour_start1               12368
peak_hour_end1                 12368
peak_hour_start2               13209
peak_hour_end2                 13209
paid_area_start_time            1831
paid_area_end_time              1831
effective_start_date               0
effective_end_date              1707
paid_parking_rate               9057
parking_category                   0
load                               0
zone                               0
weekday_rate1                   4677
weekday_start1                  4677
weekday_end1                    4677
weekday_rate2                   4677
weekday_start2                  4677
weekday_end2                    4677
weekday_rate3                   7532
weekday_start3                  7532
weekday_end3                    7532
start_time_weekday              6480
e

Dado que hay algunos casos como el anterior donde hay divergencia en la información de plazas asociadas a un mismo element_key, calculamos la media redondeada a un valor entero de los distintos valores existentes.

In [44]:
park_spaces = blocks[['element_key','parking_spaces','paid_parking_area']]
park_spaces = park_spaces.dropna()
park_spaces = park_spaces.groupby(['element_key','paid_parking_area'])['parking_spaces'].max()

In [45]:
park_spaces = park_spaces.reset_index()

In [46]:
park_spaces.head()

,element_key,paid_parking_area,parking_spaces
0,1001,Pioneer Square,7.0
1,1002,Pioneer Square,9.0
2,1005,Commercial Core,8.0
3,1006,Commercial Core,6.0
4,1009,Commercial Core,5.0


In [47]:
park_spaces[['element_key','paid_parking_area']].duplicated().sum()

0

In [48]:
park_spaces.shape

(1709, 3)

In [49]:
park_spaces[park_spaces.element_key.duplicated(keep=False)]
# Encontramos dos element key con información ambigua del distrito asociado
# Los analizaremos en el siguiente apartado ubicándolos en un mapa

,element_key,paid_parking_area,parking_spaces
194,9762,Commercial Core,4.0
195,9762,Pioneer Square,4.0
1439,78114,Belltown,15.0
1440,78114,Commercial Core,17.0


In [50]:
park_spaces.parking_spaces.describe().astype(str)

count               1709.0
mean     8.537741369221767
std      5.441127087493119
min                    0.0
25%                    5.0
50%                    8.0
75%                   10.0
max                   63.0
Name: parking_spaces, dtype: object

In [51]:
park_spaces.parking_spaces.sort_values(ascending=False).head(10)  
# el valor de 63 podría ser outlier
# comprobamos que no influye en el análisis porque ese EK no está en el fichero de coordenadas

1161    63.0
1685    50.0
754     48.0
1683    47.0
753     45.0
1673    40.0
536     39.0
919     36.0
1682    36.0
1365    34.0
Name: parking_spaces, dtype: float64

In [52]:
park_spaces.element_key.loc[park_spaces.parking_spaces == 0].count()

6

In [53]:
# Eliminamos los registros que no tienen información de la capacidad de plazas
park_spaces = park_spaces.loc[park_spaces.parking_spaces != 0]

In [54]:
park_spaces.head()

,element_key,paid_parking_area,parking_spaces
0,1001,Pioneer Square,7.0
1,1002,Pioneer Square,9.0
2,1005,Commercial Core,8.0
3,1006,Commercial Core,6.0
4,1009,Commercial Core,5.0


## Extracción de datos de ubicación geográfica

In [55]:
coord = pd.read_csv('./data/Coord_EK.csv')

In [56]:
coord.head()

,element_key,latitude,longitude
0,1001,47.602862,-122.334703
1,1002,47.602997,-122.334538
2,1005,47.603602,-122.335382
3,1006,47.603725,-122.335171
4,1009,47.605010,-122.336669


In [57]:
coord.shape

(1517, 3)

In [58]:
coord.duplicated().sum()  # no hay registros duplicados

0

In [59]:
coord.element_key.duplicated().sum()  # tampoco hay duplicados de Element Key

0

In [60]:
coord[['latitude','longitude']].duplicated().sum()  # ni hay ubicaciones repetidas

0

#### Análisis de ubicación de los element_key duplicados en park_spaces

In [61]:
coord.loc[(coord.element_key == 9762) | (coord.element_key == 78114)]

,element_key,latitude,longitude
175,9762,47.602669,-122.336182
1270,78114,47.609074,-122.343332


Utilizamos la web https://www.coordenadas-gps.com para analizar las ubicaciones y decidimos asociar ambos segmentos de calle con el distrito Commercial Core:

In [62]:
park_spaces = park_spaces[(park_spaces.index != 195) & (park_spaces.index != 1439)]  # eliminamos los duplicados

## Relación de transacciones, ubicaciones y capacidad

In [63]:
df.element_key.unique().size

1514

In [64]:
coord.element_key.unique().size

1517

In [65]:
park_spaces.element_key.unique().size

1701

In [66]:
len(set(coord.element_key).intersection(set(df.element_key)))

1445

In [67]:
len(set(park_spaces.element_key).intersection(set(coord.element_key)))

1498

**Mezclamos los datasets:**

In [68]:
# Transacciones con Coordenadas
df_c = pd.merge(df, coord, on='element_key', how='inner', validate='many_to_one')

In [69]:
# Transacciones con Coordenadas y con Capacidad de Plazas disponibles
df_cp = pd.merge(df_c, park_spaces, on='element_key', how='inner', validate='many_to_one')

In [70]:
df.shape, df_c.shape, df_cp.shape

((10855107, 4), (10690446, 6), (10685053, 8))

In [71]:
df_cp.head()

,element_key,final_date_time,transaction_date_time,transaction_id,latitude,longitude,paid_parking_area,parking_spaces
0,35693,2016-01-02 10:00:00,2016-01-02 08:00:00,64059668,47.619158,-122.346457,Uptown Triangle,10.0
1,35693,2016-01-02 10:00:00,2016-01-02 08:00:00,64059669,47.619158,-122.346457,Uptown Triangle,10.0
2,35693,2016-01-02 14:00:00,2016-01-02 08:00:00,64084672,47.619158,-122.346457,Uptown Triangle,10.0
3,35693,2016-01-02 09:00:00,2016-01-02 08:00:00,183563751,47.619158,-122.346457,Uptown Triangle,10.0
4,35693,2016-01-02 10:00:00,2016-01-02 08:00:00,64084673,47.619158,-122.346457,Uptown Triangle,10.0


In [72]:
df_cp.element_key.unique().size

1443

## Construcción de la serie de ocupación por timestamp

Construimos primero la tabla corta que usaremos para montar las series temporales:

In [73]:
in_cols = ['element_key', 'latitude', 'longitude', 'transaction_date_time', 'parking_spaces','paid_parking_area']
out_cols = ['element_key', 'latitude', 'longitude', 'final_date_time', 'parking_spaces','paid_parking_area']

df_in = df_cp[in_cols].reset_index(drop=True)
df_in['timestamp_sign'] = 1.0
df_in.rename(columns={'transaction_date_time': 'timestamp'}, inplace=True)

df_out = df_cp[out_cols].reset_index(drop=True)
df_out['timestamp_sign'] = - 1.0
df_out.rename(columns={'final_date_time': 'timestamp'}, inplace=True)  # inplace = True, value of copy is ignored

In [74]:
df_in.head()

,element_key,latitude,longitude,timestamp,parking_spaces,paid_parking_area,timestamp_sign
0,35693,47.619158,-122.346457,2016-01-02 08:00:00,10.0,Uptown Triangle,1.0
1,35693,47.619158,-122.346457,2016-01-02 08:00:00,10.0,Uptown Triangle,1.0
2,35693,47.619158,-122.346457,2016-01-02 08:00:00,10.0,Uptown Triangle,1.0
3,35693,47.619158,-122.346457,2016-01-02 08:00:00,10.0,Uptown Triangle,1.0
4,35693,47.619158,-122.346457,2016-01-02 08:00:00,10.0,Uptown Triangle,1.0


In [75]:
df_out.head()

,element_key,latitude,longitude,timestamp,parking_spaces,paid_parking_area,timestamp_sign
0,35693,47.619158,-122.346457,2016-01-02 10:00:00,10.0,Uptown Triangle,-1.0
1,35693,47.619158,-122.346457,2016-01-02 10:00:00,10.0,Uptown Triangle,-1.0
2,35693,47.619158,-122.346457,2016-01-02 14:00:00,10.0,Uptown Triangle,-1.0
3,35693,47.619158,-122.346457,2016-01-02 09:00:00,10.0,Uptown Triangle,-1.0
4,35693,47.619158,-122.346457,2016-01-02 10:00:00,10.0,Uptown Triangle,-1.0


In [76]:
full_transactions = pd.concat([df_in, df_out])
full_transactions = full_transactions.sort_values('timestamp', ascending=True)
full_transactions = full_transactions.reset_index(drop=True)

In [77]:
full_transactions.head()

,element_key,latitude,longitude,timestamp,parking_spaces,paid_parking_area,timestamp_sign
0,35693,47.619158,-122.346457,2016-01-02 08:00:00,10.0,Uptown Triangle,1.0
1,1026,47.614539,-122.348290,2016-01-02 08:00:00,8.0,Belltown,1.0
2,1026,47.614539,-122.348290,2016-01-02 08:00:00,8.0,Belltown,1.0
3,1026,47.614539,-122.348290,2016-01-02 08:00:00,8.0,Belltown,1.0
4,1026,47.614539,-122.348290,2016-01-02 08:00:00,8.0,Belltown,1.0


In [78]:
df_in.shape, df_out.shape, full_transactions.shape

((10685053, 7), (10685053, 7), (21370106, 7))

#### Añadimos porcentaje de ocupación

In [79]:
# COMANDO LARGA DURACIÓN
full_transactions['occupation'] = full_transactions.groupby(['element_key', 'timestamp']).timestamp_sign.transform(
    lambda series: series.cumsum())

In [80]:
full_transactions = full_transactions.reset_index(drop=True)

In [81]:
full_transactions.drop_duplicates(subset=['element_key','timestamp'], keep='last', inplace=True)

In [82]:
full_transactions['day_year'] = full_transactions.timestamp.dt.dayofyear

In [83]:
full_transactions['occu_cum'] = full_transactions.groupby(['element_key','day_year']).occupation.agg('cumsum')

In [84]:
full_transactions['occupation_perc'] = (full_transactions.occu_cum / full_transactions.parking_spaces * 100.0).map(
    "{0:.2f}".format).astype(float)

In [85]:
full_transactions.element_key.loc[full_transactions.occupation_perc > 100].count()

204768

In [86]:
full_transactions.element_key.loc[full_transactions.occupation_perc > 100].unique().size

1174

In [87]:
full_transactions.shape

(4138332, 11)

## Corrección de la capacidad de plazas disponibles

In [88]:
new_park_spaces = pd.read_csv('./data/StreetParking.csv')

In [89]:
new_park_spaces.head()

,element_key,parking_category,total_nopark,total_zones,parking_spaces,total_spaces
0,32018,Paid Parking,8,1,6,15
1,1042,No Parking Allowed,5,1,0,6
2,48129,Paid Parking,6,1,4,11
3,47962,Paid Parking,8,2,7,17
4,47966,Paid Parking,5,1,4,10


In [90]:
# Eliminamos los registros que no tienen información de la capacidad de plazas
new_park_spaces = new_park_spaces.loc[new_park_spaces.total_spaces != 0]

In [91]:
new_park_spaces = pd.merge(new_park_spaces, park_spaces, on='element_key', how='inner', validate='many_to_one')

In [92]:
new_park_spaces.head()

,element_key,parking_category,total_nopark,total_zones,parking_spaces_x,total_spaces,paid_parking_area,parking_spaces_y
0,32018,Paid Parking,8,1,6,15,Belltown,6.0
1,48129,Paid Parking,6,1,4,11,Pioneer Square,6.0
2,47962,Paid Parking,8,2,7,17,Commercial Core,8.0
3,47966,Paid Parking,5,1,4,10,Commercial Core,9.0
4,8617,Paid Parking,7,2,4,13,Denny Triangle,4.0


In [93]:
new_park_spaces.element_key.loc[new_park_spaces.parking_spaces_x == new_park_spaces.parking_spaces_y].count()

1055

In [94]:
new_park_spaces.shape

(1694, 8)

In [95]:
new_park_spaces = new_park_spaces.drop(columns=['total_nopark','total_zones','parking_spaces_x','parking_spaces_y'])

In [96]:
new_park_spaces.total_spaces.value_counts()

16    151
15    138
14    133
17    125
12    114
13    114
11    107
18     87
10     85
19     62
8      56
9      53
21     40
22     39
23     39
7      37
6      36
20     35
4      31
5      24
24     22
3      21
25     19
2      18
26     14
1      12
28     10
33     10
27      9
29      7
31      7
35      7
32      4
34      3
38      3
30      3
41      3
40      2
37      2
36      2
44      2
50      2
51      1
53      1
43      1
42      1
39      1
54      1
Name: total_spaces, dtype: int64

In [97]:
# Eliminamos aquellos parquímetros con un número de plazas inferior a 4
new_park_spaces = new_park_spaces.loc[new_park_spaces.total_spaces > 4]

In [98]:
df_cnp = pd.merge(df_c, new_park_spaces, on='element_key', how='inner', validate='many_to_one')

In [99]:
df_cnp.shape

(10658334, 9)

In [100]:
df_cnp.element_key.unique().size  # se corrige la capacidad de plazas para 1436-1055 = 381 element keys

1436

In [101]:
new_park_spaces = new_park_spaces.drop(columns=['paid_parking_area'])
full_transactions = full_transactions.drop(columns=['occupation_perc','timestamp_sign','occupation','parking_spaces'])
full_transactions = pd.merge(full_transactions, new_park_spaces, on='element_key', how='inner', validate='many_to_one')
full_transactions['occupation_perc'] = (full_transactions.occu_cum / full_transactions.total_spaces * 100.0).map(
    "{0:.2f}".format).astype(float)

In [102]:
full_transactions.element_key.loc[full_transactions.occupation_perc > 100].count()

9826

In [103]:
full_transactions.element_key.loc[full_transactions.occupation_perc > 100].unique().size

187

In [104]:
# Eliminamos los registros asociados a element_keys con ocupación > 100%
ek_del = full_transactions.element_key.loc[full_transactions.occupation_perc > 100].unique()
full_transactions = full_transactions.loc[~full_transactions.element_key.isin(ek_del)]

In [105]:
full_transactions.element_key.unique().size

1249

In [106]:
full_transactions.parking_category.value_counts()/full_transactions.shape[0]

Paid Parking               0.933939
Restricted Parking Zone    0.046722
Carpool Parking            0.011140
No Parking Allowed         0.008199
Name: parking_category, dtype: float64

In [107]:
# Nos quedamos con la categoría mayoritaria Paid Parking
full_transactions = full_transactions.loc[full_transactions.parking_category == 'Paid Parking']

In [108]:
# Comprobamos que la ocupación es 0 para las transacciones con hora = 20
full_transactions.occupation_perc.loc[full_transactions.timestamp.dt.hour == 20].unique()

array([0.])

In [109]:
# Y las eliminamos
full_transactions = full_transactions.loc[full_transactions.timestamp.dt.hour < 20]

In [110]:
full_transactions = full_transactions.drop(columns=['occu_cum','parking_category','total_spaces'])

In [111]:
full_transactions.head()

,element_key,latitude,longitude,timestamp,paid_parking_area,day_year,occupation_perc
0,85382,47.680345,-122.325125,2016-01-02 08:00:00,Green Lake,2,13.33
1,85382,47.680345,-122.325125,2016-01-02 09:00:00,Green Lake,2,33.33
2,85382,47.680345,-122.325125,2016-01-02 10:00:00,Green Lake,2,26.67
3,85382,47.680345,-122.325125,2016-01-02 11:00:00,Green Lake,2,60.00
4,85382,47.680345,-122.325125,2016-01-02 12:00:00,Green Lake,2,40.00


In [112]:
full_transactions.shape

(3143796, 7)

In [116]:
full_transactions.to_csv('./data/FullTrans016_before_ext', index=False)

In [113]:
df_cnp = df_cnp.loc[df_cnp.parking_category == 'Paid Parking']
df_cnp = df_cnp.drop(columns=['transaction_id','parking_category'])
df_cnp = df_cnp.loc[~df_cnp.element_key.isin(ek_del)]
df_cnp = df_cnp.reset_index(drop=True)

In [114]:
df_cnp.head()

,element_key,final_date_time,transaction_date_time,latitude,longitude,total_spaces,paid_parking_area
0,53549,2016-01-02 10:00:00,2016-01-02 08:00:00,47.628175,-122.341132,50,South Lake Union
1,53549,2016-01-02 17:00:00,2016-01-02 08:00:00,47.628175,-122.341132,50,South Lake Union
2,53549,2016-01-02 09:00:00,2016-01-02 08:00:00,47.628175,-122.341132,50,South Lake Union
3,53549,2016-01-02 09:00:00,2016-01-02 08:00:00,47.628175,-122.341132,50,South Lake Union
4,53549,2016-01-02 09:00:00,2016-01-02 08:00:00,47.628175,-122.341132,50,South Lake Union


In [115]:
# Guardamos las transacciones asociadas con ubicaciones y capacidad de plazas
df_cnp.to_csv('./data/ParkingTrans2016_with_Coord_Spaces_conhol.csv', index=False)

## Completamos la serie con las horas intermedias faltantes

In [140]:
full_transactions = pd.read_csv('./data/FullTrans016_before_ext', parse_dates=['timestamp'])

In [141]:
anyo16 = pd.date_range(start='2016-01-01', periods=366*24, freq='H')
anyo16 = pd.DataFrame(data=anyo16.to_pydatetime())
anyo16.columns = ['timestamp']
anyo16 = anyo16.loc[(anyo16.timestamp.dt.hour >= 8) & (anyo16.timestamp.dt.hour < 20)]
anyo16['day_year'] = anyo16.timestamp.dt.dayofyear
anyo16['hour'] = anyo16.timestamp.dt.hour

anyo16_eks = pd.DataFrame()
all_eks = sorted(full_transactions.element_key.unique())
for ek in all_eks:
    anyo16['element_key'] = ek
    anyo16_eks = pd.concat([anyo16_eks, anyo16], sort=True)

full_transactions['hour'] = full_transactions.timestamp.dt.hour
full_transactions = full_transactions.drop(columns=['timestamp', 'latitude', 'longitude', 'paid_parking_area'])
ft_ext = pd.merge(anyo16_eks, full_transactions, on=['day_year', 'hour', 'element_key'],
                           how = 'left', validate='many_to_one')

In [144]:
ft_ext = ft_ext.sort_values(['element_key','timestamp'])

In [145]:
ft_ext.loc[ft_ext.hour == 8] = ft_ext.loc[ft_ext.hour == 8].fillna(0)

In [162]:
ft_ext = ft_ext.ffill()

In [168]:
ft_ext = ft_ext.loc[ft_ext.timestamp.dt.weekday != 6]
ft_ext = pd.merge(ft_ext, coord, on='element_key', how='left', validate='many_to_one')
ft_ext = pd.merge(ft_ext, park_spaces, on='element_key', how='left', validate='many_to_one')

In [170]:
ft_ext = ft_ext.drop(columns=['parking_spaces'])

In [171]:
ft_ext.head()

,day_year,element_key,hour,timestamp,occupation_perc,latitude,longitude,paid_parking_area
0,1,1001,8,2016-01-01 08:00:00,0.0,47.602862,-122.334703,Pioneer Square
1,1,1001,9,2016-01-01 09:00:00,0.0,47.602862,-122.334703,Pioneer Square
2,1,1001,10,2016-01-01 10:00:00,0.0,47.602862,-122.334703,Pioneer Square
3,1,1001,11,2016-01-01 11:00:00,0.0,47.602862,-122.334703,Pioneer Square
4,1,1001,12,2016-01-01 12:00:00,0.0,47.602862,-122.334703,Pioneer Square


In [172]:
ft_ext.to_csv('./data/FullTrans2016_ext_conhol.csv', index=False)

## Completamos la serie con variables adicionales

#### Datos meteorológicos globales

In [173]:
meteo = pd.read_csv('./data/seattleWeather_1948-2017.csv', parse_dates=['DATE'])
meteo.columns = meteo.columns.map(lambda x: camelToSnake(x))

meteo = meteo.loc[meteo['date'].dt.year == 2016]
meteo['day_year'] = meteo.date.dt.dayofyear

meteo.tmax = ((meteo.tmax - 32) * 5.0/9.0).map("{0:.2f}".format).astype(float)
meteo.tmin = ((meteo.tmin - 32) * 5.0/9.0).map("{0:.2f}".format).astype(float)

In [174]:
meteo.head()

,date,prcp,tmax,tmin,rain,day_year
24837,2016-01-01,0.00,7.78,-2.22,False,1
24838,2016-01-02,0.00,5.56,-3.89,False,2
24839,2016-01-03,0.02,4.44,-0.56,True,3
24840,2016-01-04,0.15,3.33,1.67,True,4
24841,2016-01-05,0.11,7.78,2.22,True,5


In [175]:
meteo.shape

(366, 6)

In [176]:
ft_ext_meteo = pd.merge(ft_ext, meteo, on='day_year', how='left', validate='many_to_one')

In [177]:
ft_ext_meteo.head()

,day_year,element_key,hour,timestamp,occupation_perc,latitude,longitude,paid_parking_area,date,prcp,tmax,tmin,rain
0,1,1001,8,2016-01-01 08:00:00,0.0,47.602862,-122.334703,Pioneer Square,2016-01-01,0.0,7.78,-2.22,False
1,1,1001,9,2016-01-01 09:00:00,0.0,47.602862,-122.334703,Pioneer Square,2016-01-01,0.0,7.78,-2.22,False
2,1,1001,10,2016-01-01 10:00:00,0.0,47.602862,-122.334703,Pioneer Square,2016-01-01,0.0,7.78,-2.22,False
3,1,1001,11,2016-01-01 11:00:00,0.0,47.602862,-122.334703,Pioneer Square,2016-01-01,0.0,7.78,-2.22,False
4,1,1001,12,2016-01-01 12:00:00,0.0,47.602862,-122.334703,Pioneer Square,2016-01-01,0.0,7.78,-2.22,False


In [178]:
ft_ext_meteo = ft_ext_meteo.drop(columns=['date','rain'])

#### Sensores de temperatura

In [179]:
weather_stations = pd.read_csv('./data/Coord_EK_stations.csv')
weather_stations.head()
weather_stations = weather_stations.drop(columns=['latitude', 'longitude'])

In [180]:
ft_ext_meteo_ws = pd.merge(ft_ext_meteo, weather_stations, on='element_key', how='left', validate='many_to_one')

In [181]:
ft_ext_meteo_ws.head()

,day_year,element_key,hour,timestamp,occupation_perc,latitude,longitude,paid_parking_area,prcp,tmax,tmin,station_closest
0,1,1001,8,2016-01-01 08:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5
1,1,1001,9,2016-01-01 09:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5
2,1,1001,10,2016-01-01 10:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5
3,1,1001,11,2016-01-01 11:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5
4,1,1001,12,2016-01-01 12:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5


In [182]:
ft_ext_meteo_ws.timestamp = pd.to_datetime(ft_ext_meteo_ws.timestamp)

In [183]:
rwis = pd.read_csv('./data/RWIS_completed.csv', parse_dates=['timestamp'])

In [184]:
rwis.head()

,station_closest,timestamp,air_temp,road_temp
0,5,2016-01-01 08:00:00,2.03,-2.63
1,5,2016-01-01 09:00:00,1.99,-2.27
2,5,2016-01-01 10:00:00,2.08,-0.89
3,5,2016-01-01 11:00:00,2.28,2.44
4,5,2016-01-01 12:00:00,2.57,4.87


In [185]:
ft_ext_meteo_ws_rwis = pd.merge(ft_ext_meteo_ws, rwis, on=['timestamp','station_closest'], how='left', 
                                validate='many_to_one')

In [186]:
ft_ext_meteo_ws_rwis.head()

,day_year,element_key,hour,timestamp,occupation_perc,latitude,longitude,paid_parking_area,prcp,tmax,tmin,station_closest,air_temp,road_temp
0,1,1001,8,2016-01-01 08:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5,2.03,-2.63
1,1,1001,9,2016-01-01 09:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5,1.99,-2.27
2,1,1001,10,2016-01-01 10:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5,2.08,-0.89
3,1,1001,11,2016-01-01 11:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5,2.28,2.44
4,1,1001,12,2016-01-01 12:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5,2.57,4.87


#### Lugares deportivos y de interés cultural

In [187]:
cult_sport = pd.read_csv('./data/Coord_cult_&_sport.csv')

In [188]:
cult_sport.head()

,element_key,latitude,longitude,poi,baseball,tennis,basket,soccer
0,1001,47.602862,-122.334703,1,0,0,0,0
1,1002,47.602997,-122.334538,1,0,0,0,0
2,1005,47.603602,-122.335382,0,0,0,0,0
3,1006,47.603725,-122.335171,0,0,0,0,0
4,1009,47.605010,-122.336669,1,0,0,0,0


In [189]:
cult_sport = cult_sport.drop(columns=['latitude', 'longitude'])
ft_ext_met_ws_rwis_cs = pd.merge(ft_ext_meteo_ws_rwis, cult_sport, on=['element_key'], 
                             how='left', validate='many_to_one')

In [190]:
ft_ext_met_ws_rwis_cs.head()

,day_year,element_key,hour,timestamp,occupation_perc,latitude,longitude,paid_parking_area,prcp,tmax,tmin,station_closest,air_temp,road_temp,poi,baseball,tennis,basket,soccer
0,1,1001,8,2016-01-01 08:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5,2.03,-2.63,1,0,0,0,0
1,1,1001,9,2016-01-01 09:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5,1.99,-2.27,1,0,0,0,0
2,1,1001,10,2016-01-01 10:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5,2.08,-0.89,1,0,0,0,0
3,1,1001,11,2016-01-01 11:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5,2.28,2.44,1,0,0,0,0
4,1,1001,12,2016-01-01 12:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,-2.22,5,2.57,4.87,1,0,0,0,0


#### Eventos

In [191]:
events = pd.read_csv('./data/Events_2016.csv')

In [192]:
events.head()

,Latitude,Longitude,day_year
0,47.611543,-122.33263,105
1,47.611543,-122.33263,106
2,47.601130,-122.32980,137
3,47.628560,-122.33979,137
4,47.636290,-122.35922,137


In [193]:
# Función para calcular la distancia entre dos puntos teniendo en cuenta la curvatura de la tierra (no Euclidea)
from math import radians, cos, sin, asin, sqrt, acos

def haversine(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6373 # Radius of earth kilometers
    return c * r

In [194]:
# Creamos una nueva columna que indica si hay un evento próximo al element_key
# Una vez que un element_key tiene un punto cercano (<=75 metros) no es necesario revisar otros (break)
ft_ext_met_ws_rwis_cs['event'] = 0
for ft_index, ft_row in ft_ext_met_ws_rwis_cs.iterrows():
    events_sub = events.loc[events.day_year == ft_row.day_year]
    for e_index, e_row in events_sub.iterrows():
        dist = haversine(e_row.Longitude, e_row.Latitude, ft_row.longitude, ft_row.latitude)
        if dist <= 0.075:
            ft_ext_met_ws_rwis_cs.at[ft_index,'event'] = 1
            break

#### Polución

In [195]:
aqds = pd.read_csv('./data/Air_Quality_Data_Seattle_2016.csv')

In [196]:
aqds.head()

,day_year,no2,co,pm2_5,o3
0,1,75.262667,858.75,26.685000,61.0
1,2,71.001333,1030.50,19.875000,83.0
2,3,72.568000,801.50,14.281250,62.0
3,4,64.860000,629.75,11.047368,49.0
4,5,70.938667,1030.50,14.912500,45.0


In [197]:
ft_ext_met_ws_rwis_cs_aq = pd.merge(ft_ext_met_ws_rwis_cs, aqds, on=['day_year'], how='left', validate='many_to_one')

In [198]:
ft_ext_met_ws_rwis_cs_aq.head()

,day_year,element_key,hour,timestamp,occupation_perc,latitude,longitude,paid_parking_area,prcp,tmax,...,poi,baseball,tennis,basket,soccer,event,no2,co,pm2_5,o3
0,1,1001,8,2016-01-01 08:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,...,1,0,0,0,0,0,75.262667,858.75,26.685,61.0
1,1,1001,9,2016-01-01 09:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,...,1,0,0,0,0,0,75.262667,858.75,26.685,61.0
2,1,1001,10,2016-01-01 10:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,...,1,0,0,0,0,0,75.262667,858.75,26.685,61.0
3,1,1001,11,2016-01-01 11:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,...,1,0,0,0,0,0,75.262667,858.75,26.685,61.0
4,1,1001,12,2016-01-01 12:00:00,0.0,47.602862,-122.334703,Pioneer Square,0.0,7.78,...,1,0,0,0,0,0,75.262667,858.75,26.685,61.0


In [199]:
ft_ext_met_ws_rwis_cs_aq.columns

Index(['day_year', 'element_key', 'hour', 'timestamp', 'occupation_perc',
       'latitude', 'longitude', 'paid_parking_area', 'prcp', 'tmax', 'tmin',
       'station_closest', 'air_temp', 'road_temp', 'poi', 'baseball', 'tennis',
       'basket', 'soccer', 'event', 'no2', 'co', 'pm2_5', 'o3'],
      dtype='object')

In [200]:
ft_ext_met_ws_rwis_cs_aq.isnull().sum()

day_year             0
element_key          0
hour                 0
timestamp            0
occupation_perc      0
latitude             0
longitude            0
paid_parking_area    0
prcp                 0
tmax                 0
tmin                 0
station_closest      0
air_temp             0
road_temp            0
poi                  0
baseball             0
tennis               0
basket               0
soccer               0
event                0
no2                  0
co                   0
pm2_5                0
o3                   0
dtype: int64

In [201]:
ft_ext_met_ws_rwis_cs_aq = ft_ext_met_ws_rwis_cs_aq.drop(columns=['station_closest'])
ft_ext_met_ws_rwis_cs_aq.to_csv('./data/Serie_Total2016_ext.csv', index=False)

In [203]:
# Comprobaciones
ft_ext_met_ws_rwis_cs_aq.shape, ft_ext_met_ws_rwis_cs.shape, ft_ext_meteo_ws_rwis.shape, ft_ext_meteo_ws.shape, ft_ext_meteo.shape, ft_ext.shape

((4182480, 23),
 (4182480, 20),
 (4182480, 14),
 (4182480, 12),
 (4182480, 11),
 (4182480, 8))

## Filtramos la serie

#### Filtrado de Element Keys con menos días sin transacciones

In [205]:
ek_day_ocu = ft_ext_met_ws_rwis_cs_aq.groupby(['element_key','day_year']).occupation_perc.sum()
ek_day_ocu = ek_day_ocu.reset_index()

In [244]:
ek_day_ocu.head()

,element_key,day_year,occupation_perc
0,1001,1,0.00
1,1001,2,14.28
2,1001,4,71.43
3,1001,5,85.72
4,1001,6,164.28


In [238]:
alldays_list = ek_day_ocu.loc[(ek_day_ocu.occupation_perc == 0)].groupby(
    'element_key').day_year.count().sort_values().head(165).index.get_values()

#### Filtrado de Element Keys con mayor número de transacciones

In [227]:
top100_list = df_cnp.groupby('element_key').total_spaces.count().sort_values(
    ascending=False).head(100).index.get_values()

#### Selección final de Element Keys 

In [239]:
len(set(top100_list) & set(alldays_list))

30

In [240]:
selected_list = list(set(top100_list) & set(alldays_list))

In [279]:
df_sel = ft_ext_met_ws_rwis_cs_aq.loc[ft_ext_met_ws_rwis_cs_aq.element_key.isin(selected_list)]

In [280]:
df_sel.head()

,day_year,element_key,hour,timestamp,occupation_perc,latitude,longitude,paid_parking_area,prcp,tmax,...,poi,baseball,tennis,basket,soccer,event,no2,co,pm2_5,o3
64056,1,1037,8,2016-01-01 08:00:00,0.0,47.621478,-122.355531,Uptown,0.0,7.78,...,0,0,0,0,0,0,75.262667,858.75,26.685,61.0
64057,1,1037,9,2016-01-01 09:00:00,0.0,47.621478,-122.355531,Uptown,0.0,7.78,...,0,0,0,0,0,0,75.262667,858.75,26.685,61.0
64058,1,1037,10,2016-01-01 10:00:00,0.0,47.621478,-122.355531,Uptown,0.0,7.78,...,0,0,0,0,0,0,75.262667,858.75,26.685,61.0
64059,1,1037,11,2016-01-01 11:00:00,0.0,47.621478,-122.355531,Uptown,0.0,7.78,...,0,0,0,0,0,0,75.262667,858.75,26.685,61.0
64060,1,1037,12,2016-01-01 12:00:00,0.0,47.621478,-122.355531,Uptown,0.0,7.78,...,0,0,0,0,0,0,75.262667,858.75,26.685,61.0


In [281]:
# Para los días festivos interpolamos
eks_sel = sorted(df_sel.element_key.unique())
hol_days = [1, 18, 46, 151, 186, 249, 316, 329, 361]
hours = list(range(8,20))

for ek in eks_sel:
    for hd in hol_days:
        for h in hours:
            i = df_sel.loc[(df_sel.element_key == ek) & (df_sel.day_year == hd) & (df_sel.hour == h)].index[0]
            if hd != 1:
                new_op1 = df_sel.occupation_perc.loc[(df_sel.element_key == ek) & (df_sel.day_year == (hd-7)) & (
                    df_sel.hour == h)].values[0]
                new_op2 = df_sel.occupation_perc.loc[(df_sel.element_key == ek) & (df_sel.day_year == (hd-14)) & (
                    df_sel.hour == h)].values[0]
            else:
                new_op1 = df_sel.occupation_perc.loc[(df_sel.element_key == ek) & (df_sel.day_year == (hd+7)) & (
                    df_sel.hour == h)].values[0]
                new_op2 = df_sel.occupation_perc.loc[(df_sel.element_key == ek) & (df_sel.day_year == (hd+14)) & (
                    df_sel.hour == h)].values[0]
            df_sel.at[i,'occupation_perc'] = (new_op1 + new_op2)/2

In [282]:
df_sel.shape

(113040, 23)

In [283]:
df_sel.element_key.loc[df_sel.occupation_perc == 0].value_counts()

62458    1257
81117     896
13793     832
13549     789
41030     679
69098     672
86449     647
1433      641
35681     632
35682     631
18622     630
85381     237
59958     209
85385     185
37134     179
76962     154
63125     136
34938     122
1045      121
9510      120
14677     115
37177     102
24557      75
1046       67
11878      63
76961      62
1037       54
79741      38
30698      31
12289      18
Name: element_key, dtype: int64

In [284]:
df_sel.element_key.loc[df_sel.occupation_perc == 0].count()/df_sel.shape[0]*100  # 9.5%

9.194975230007078

In [285]:
df_sel.head()

,day_year,element_key,hour,timestamp,occupation_perc,latitude,longitude,paid_parking_area,prcp,tmax,...,poi,baseball,tennis,basket,soccer,event,no2,co,pm2_5,o3
64056,1,1037,8,2016-01-01 08:00:00,5.770,47.621478,-122.355531,Uptown,0.0,7.78,...,0,0,0,0,0,0,75.262667,858.75,26.685,61.0
64057,1,1037,9,2016-01-01 09:00:00,7.695,47.621478,-122.355531,Uptown,0.0,7.78,...,0,0,0,0,0,0,75.262667,858.75,26.685,61.0
64058,1,1037,10,2016-01-01 10:00:00,9.615,47.621478,-122.355531,Uptown,0.0,7.78,...,0,0,0,0,0,0,75.262667,858.75,26.685,61.0
64059,1,1037,11,2016-01-01 11:00:00,13.465,47.621478,-122.355531,Uptown,0.0,7.78,...,0,0,0,0,0,0,75.262667,858.75,26.685,61.0
64060,1,1037,12,2016-01-01 12:00:00,19.230,47.621478,-122.355531,Uptown,0.0,7.78,...,0,0,0,0,0,0,75.262667,858.75,26.685,61.0


In [286]:
df_sel.to_csv('./data/SerieTotal2016_ext_selected.csv', index=False)